In [ ]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
input_data = Path("../matched-infoboxes-extracted/")
inp = list(input_data.rglob('*.json'))
files = [x for x in inp if x.is_file()]
len(files) # total 580

In [ ]:
num_edits = 0
change_tuples = []
for file in tqdm(files[:3]):
    # print(file)
    with open(file, 'r', encoding='utf-8') as f:
        for jsonObj in f:
            single_edit = json.loads(jsonObj)
            num_edits += 1
            #entries.append(single_edit)
            title = single_edit['pageTitle']
            changes = single_edit['changes']
            timestamp = single_edit['validFrom']
            revisionId = single_edit['revisionId']
            # print(single_edit['user'])
            user_name = single_edit['user']['username'] if 'username' in single_edit['user'].keys() else None
            user_id = single_edit['user']['id'] if 'id' in single_edit['user'].keys() else None
            user_ip = single_edit['user']['ip'] if 'ip' in single_edit['user'].keys() else None
            for change in changes:
                name = change['property']['name']
                current_value = change['currentValue'] if 'currentValue' in change.keys() else None
                previous_value = change['previousValue'] if 'previousValue' in change.keys() else None
                change_tuples.append((title, name, previous_value, current_value, timestamp, revisionId, user_name, user_id, user_ip))
print(num_edits) # 1934309 for 50
len(change_tuples) # 9715201 for 50

In [ ]:
data = pd.DataFrame(change_tuples, columns=['title', 'name', 'previous_value', 'current_value', 'timestamp', 'revisionId', 'user_name', 'user_id', 'user_ip'])
data['timestamp'] = pd.to_datetime(data['timestamp'])

# Analyze number of changes

In [ ]:
changes_per_page = data.groupby(['title'])['timestamp'].count()
plt.hist(changes_per_page.to_numpy(), bins=100)
plt.yscale('log')
plt.title("Number of changes per page")
plt.ylabel("#Occurances, log")

In [ ]:
changes_per_attribute = data.groupby(['title', 'name'])['timestamp'].count()
plt.hist(changes_per_attribute.to_numpy(), bins=100)
plt.yscale('log')
plt.title("Number of changes per attribute")
plt.ylabel("#Occurances, log")

In [ ]:
data_with_agg = data.join(changes_per_page, on='title', rsuffix='_r').rename(columns={'timestamp_r': 'changes_per_page'})
data_with_agg = data_with_agg.join(changes_per_attribute, on=['title', 'name'], rsuffix='_r').rename(columns={'timestamp_r': 'changes_per_attribute'})

In [ ]:
data_with_agg.sort_values('changes_per_attribute')